In [13]:
import io
import requests
import pandas as pd
from simplenlg.lexicon import Lexicon
from simplenlg.realiser.english import Realiser
from simplenlg.framework import NLGFactory
from simplenlg.phrasespec import NPPhraseSpec, SPhraseSpec

In [14]:
url = 'https://raw.githubusercontent.com/dadejecova/Natural-Language-Python/main/dataset/financials.csv'
response = requests.get(url)

In [15]:
if response.status_code == 200:
  #we read this with pnds
  stocks_df = pd.read_csv(io.StringIO(response.text))
  print(stocks_df.sample(10))
else:
  print("Failed to fetch Datset")

    Symbol                        Name                      Sector   Price  \
147  DISCA  Discovery Communications-A      Consumer Discretionary   22.87   
106    CMG      Chipotle Mexican Grill      Consumer Discretionary  266.01   
126    COP              ConocoPhillips                      Energy   53.24   
486    HCN              Welltower Inc.                 Real Estate   54.67   
430    TPR              Tapestry, Inc.      Consumer Discretionary   48.85   
242   INFO             IHS Markit Ltd.                 Industrials   43.73   
474     VZ      Verizon Communications  Telecommunication Services   49.04   
103   CHTR      Charter Communications      Consumer Discretionary  348.65   
252    IFF   Intl Flavors & Fragrances                   Materials  138.00   
236    HST       Host Hotels & Resorts                 Real Estate   18.75   

     Price/Earnings  Dividend Yield  Earnings/Share  52 Week Low  \
147           11.00        0.000000            1.96       30.250   
106  

In [19]:
stocks_df.describe()

,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book
count,505.000000,503.000000,505.000000,505.000000,505.000000,505.000000,5.050000e+02,5.050000e+02,505.000000,497.000000
mean,103.830634,24.808390,1.895953,3.753743,122.623832,83.536616,4.923944e+10,3.590328e+09,3.941705,14.453179
std,134.427636,41.241081,1.537214,5.689036,155.362140,105.725473,9.005017e+10,6.840544e+09,3.460110,89.660508
min,2.820000,-251.530000,0.000000,-28.010000,6.590000,2.800000,2.626102e+09,-5.067000e+09,0.153186,0.510000
25%,46.250000,15.350000,0.794834,1.490000,56.250000,38.430000,1.273207e+10,7.739320e+08,1.629490,2.020000
50%,73.920000,19.450000,1.769255,2.890000,86.680000,62.850000,2.140095e+10,1.614399e+09,2.896440,3.400000
75%,116.540000,25.750000,2.781114,5.140000,140.130000,96.660000,4.511968e+10,3.692749e+09,4.703842,6.110000
max,1806.060000,520.150000,12.661196,44.090000,2067.990000,1589.000000,8.095080e+11,7.938600e+10,20.094294,1403.380000


In [20]:
lexicon = Lexicon.getDefaultLexicon()

realiser = Realiser(lexicon)

nlgFactory = NLGFactory(lexicon)

In [ ]:
def create_descriptions(row):

    subject = nlgFactory.createNounPhrase(
        "The company " + 
        str(row['Name'])
    )

    verb1 = nlgFactory.createVerbPhrase(
        "is part of the " + 
        str(row['Sector']) + 
        " Sector"
    )

    object1 = nlgFactory.createNounPhrase(
        " and is currently trading at $" + 
        str(row['Price']) + " per share."
    )

    clause1 = nlgFactory.createClause(subject, verb1, object1)

    verb2 = nlgFactory.createVerbPhrase(
        "Boasts"
    )

    object2 = nlgFactory.createNounPhrase(
        "a market capitalization of " + 
        str(round(row['Market Cap']/1e+9, 2)) + 
        " billion dollars."
    )

    clause2 = nlgFactory.createClause(
        "the company ", verb2, object2
    )

    object3 = nlgFactory.createNounPhrase(
        " The annual dividend yield is " + 
        str(round(row['Dividend Yield'], 2))
    )

    clause3 = nlgFactory.createClause(
        object3
    )

    verb4 = nlgFactory.creteVerbPhrase(
        "have"
    )

    object4 = nlgFactory.createNounPhrase(
        "a strong financial position with an EBITDA of " +
        str(round(row['EBITDA']/1e+9, 2)) + 
        " billion dollars."
    )

    clause4 = nlgFactory.createClause(
        "It", verb4, object4
    )

    if row['EBITDA'] < 0:
        # This is to negate the clause
        # If the EBITDA is negative, we want to say that the company does not have a strong financial position
        clause4.setFeature(Feature.NEGATED, True)


    s1 = nlgFactory.createSentence(clause1)
    s2 = nlgFactory.createSentence(clause2)
    s3 = nlgFactory.createSentence(clause3)
    s4 = nlgFactory.createSentence(clause4)

    paragraph = nlgFactory.createParagraph([
        s1, s2, s3, s4
    ])

    output = realiser.realise(paragraph).getRealisation()

    return output.strip()


In [22]:
stocks_df['text'] = stocks_df.apply(lambda r: create_descriptions(r), axis=1)

stocks_df[['Name', 'Sector', 'Price', 'Dividend Yield', 'Market Cap', 'text']].sample(10)

AttributeError: 'NLGFactory' object has no attribute 'creteClause'